# Análise das Aposentadorias por Idade

Este documento apresenta uma descrição dos dados de aposentadoria por idade do Regime Geral de Previdência Social (RGPS) no período de 1995 a 2016.

## Filtrando os dados

Inicialmente, vamos obter todos os dados de aposentadoria por idade do banco de dados.

Nos dados obtidos, existem vários tipos de aposentadoria por idade, conforme apresentado abaixo:

In [3]:
# Carrega as variáveis do arquivo de dicionário
%run util/dicionario_dados.py

print("CÓDIGO\t BENEFÍCIO")
print("-"*20)

# variável que salva os codigos das aposentadorias
codigos_apos_id = []

# A varíavel codigos_especie (criada pelo comando "%run util/dicionario_dados.py") é um dicionário com pares de código e benefícios
# Itera sobre os valores do dicionário filtrando as aposentadorias por idade
for chave, valor in codigos_especie.items():
    if 'idade' in valor and 'Apos' in valor:
        print("{}\t {}".format(chave,valor))
        codigos_apos_id.append(chave)

CÓDIGO	 BENEFÍCIO
--------------------
7	 Aposentadoria por idade do trab. rural
8	 Aposentadoria por idade do emp. rural
41	 Aposentadoria por idade
52	 Aposentadoria por idade (Extinto Plano Básico)
78	 Aposentadoria por idade de ex-combatente marítimo
81	 Aposentadoria por idade compulsória (Ex-SASSE)


Assim, temos 6 tipos de aposentadorias no banco de dados.

Agora vamos verificar quantos registros existem para cada um dos tipos de aposentadoria.

In [2]:
# importa os pacotes que serão utilizados
import pandas as pd
import sqlite3

# Função que executa as queries no banco de dados
def run_query(q):
    with sqlite3.connect('microdados/sqlite3/microdadosRGPS.db') as conn:
        return pd.read_sql(q, conn)    

In [ ]:
# Obtém a quantidade de registros por tipo de aposentadoria
run_query('SELECT ESPECIE as CODIGO, COUNT(ESPECIE) as QTD_REGISTROS FROM apos_aux WHERE ESPECIE IN (7,8,41,52,78,81) GROUP BY ESPECIE') 

Conforme observado, no banco de dados só existem registros do tipo 41 (Aposentadoria por idade). Esse fato também é apresentado no documento *analise-aposentadoria-idade.ipynb* deste projeto.

### Limpeza dos dados

A primeira etapa da limpeza é verificar a quantidade de benefícios por situação (ATIVO, CESSADO, SUSPENSO etc.).

In [15]:
# Realiza a Query no banco de dados
reg_situacao = run_query("SELECT SITUACAO AS CODIGO, COUNT(SITUACAO) AS QUANTIDADE FROM apos_aux WHERE ESPECIE=41 GROUP BY SITUACAO ORDER BY QUANTIDADE DESC")
# Substitui os códigos da coluna SITUACAO pelo texto correspondente
reg_situacao['SITUACAO'] = reg_situacao['CODIGO'].apply(lambda x: codigos_situacao[x])
# Adiciona mais uma coluna com a porcentagem relativa em relação ao total
reg_situacao['PROPORCAO (%)'] = 100*(reg_situacao['QUANTIDADE']/reg_situacao['QUANTIDADE'].sum())
reg_situacao

,CODIGO,QUANTIDADE,SITUACAO,PROPORCAO (%)
0,0,8785320,ATIVO,83.129169
1,8,1345235,CESSADO PELO SISOBI,12.728992
2,2,406853,CESSADO,3.849758
3,3,18156,SUSPENSO,0.171797
4,7,5033,SUSPENSO PELO CONPAG,0.047624
5,23,4073,SUSPENSO PELO SISOBI,0.038540
6,16,1831,SUSPENSO PELA AUDITORIA,0.017325
7,18,1268,CESSADO PELA AUDITORIA,0.011998
8,17,243,CESSADO PELA INSPETORIA,0.002299
9,24,196,CANCELADO PELA AUDITORIA,0.001855


Pelos os dados expostos na tabela acima, 83% dos benefícios estão ativos e cerca de 16% estão cessados. 

Para facilitar as análises seguintes, os seguintes agrupamentos serão realizados:

* CESSADO PELA INSPETORIA, CESSADO PELA AUDITORIA, CESSADO PELO SISOBI e CESSADO serão do tipo **CESSADO**
* SUSPENSO, SUSPENSO PELO CONPAG, SUSPENSO PELO SISOBI e SUSPENSO PELA AUDITORIA serão do tipo **SUSPENSO**

Os benefícios abaixo serão desconsiderados:
* CANCELADO PELA AUDITORIA
* CESSADO REVISAO RURAL/95
* SUSPENSO POR MARCA DE ERRO

Por fim, nosso dados terão somente 3 tipos de situações: **ATIVOS, CESSADOS e SUSPENSOS.**

In [4]:
# Carrega dos dados
apos_id = run_query("SELECT * FROM apos_aux WHERE ESPECIE=41")

# Converte algumas colunas para o tipo "category" para reduzir o uso de memória
# Reduz de 1.18 Gb para 0.8 Gb
colunas = ['ESPECIE', 'MOT_CESSACAO', 'CLIENTELA', 'SEXO', 'SITUACAO']
for c in colunas:
    apos_id[c] = apos_id[c].astype('category')
    
    

In [6]:
# Remove os dados com situação do tipo 'CANCELADO PELA AUDITORIA', 'CESSADO REVISAO RURAL/95' e 'SUSPENSO POR MARCA DE ERRO'
for cod_situacao in [24,22,4]:
    apos_id = apos_id[apos_id['SITUACAO'] != cod_situacao]

# Agrupa os dados da coluna SITUACAO
apos_id['SITUACAO'] == ''


In [13]:
apos_id[apos_id['SITUACAO'] != 4]

,ESPECIE,DIB,DDB,MOT_CESSACAO,ULT_COMPET_MR,VL_MR,DT_NASC,VL_RMI,CLIENTELA,SEXO,SITUACAO,DT_OBITO,IDADE_DIB,TEMPO_CONTRIB
1928577,41,07/09/2000,21/09/2000,8,201208,622,05/05/1933,151,2,1,4,None,67,14
3086187,41,11/03/2003,24/03/2003,53,201703,937,08/03/1948,200,2,3,4,None,55,22
3121197,41,28/09/2016,28/09/2005,53,201612,880,03/09/1949,880,2,3,4,None,67,15
5533942,41,02/07/2007,19/07/2007,53,201508,"756,81",27/06/1942,"473,62",1,1,4,None,65,25
5620507,41,21/01/2009,19/03/2009,53,201610,"673,19",28/12/1948,415,1,3,4,None,60,16
5981784,41,02/11/2008,27/06/2008,53,201103,545,02/11/1948,415,2,1,4,None,60,20
6471337,41,09/09/2009,11/09/2009,53,201610,"876,88",11/06/1941,465,1,1,4,None,68,14
6507210,41,28/06/2010,12/07/2010,53,201508,"662,74",28/05/1944,510,1,1,4,None,66,14
6750018,41,08/09/2010,15/10/2010,53,201610,"739,36",04/09/1942,510,1,1,4,None,68,0
7211983,41,13/04/2016,01/06/2011,53,201611,880,01/01/1952,880,2,3,4,None,64,0


## Quantidade de concessões por ano

O próximo passo na análise é verificar a quantidade de aposentadorias por idade concedidas a cada ano. 

In [ ]:
# Idade média de aposentadoria por ano